In [9]:
# Cell 1: imports & params
import os
import numpy as np
import pandas as pd
from scipy.signal import spectrogram
import joblib
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, callbacks
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold, StratifiedKFold, train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, precision_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

# PARAMETERS (edit these if needed)
DATASET_FOLDER = "dataset"      # folder with your training files (excel/csv)
DATA_COLUMNS = ["ax","ay","az","gx","gy","gz"]  # expected columns in data files
WINDOW_SIZE = 64
STRIDE = WINDOW_SIZE // 2
FS = 100                       # sampling frequency estimate used for spectrogram
SPEC_NPERSEG = 32
SPEC_NOVERLAP = 16
BATCH_SIZE = 64
EPOCHS = 60
LEARNING_RATE = 1e-3
DROPOUT_RATE = 0.35
L2_REG = 1e-4
N_FOLDS = 5
SCALER_PATH = "scaler_1.pkl"
MODEL_SAVE_PATH = "hybrid_model.keras"
ACC_CHANNELS = ["ax","ay","az"]

In [11]:
# Cell 2: discover dataset files and assign labels (auto-label if explicit map missing)
print("Looking for dataset files in:", DATASET_FOLDER)
found_files = []
for f in os.listdir(DATASET_FOLDER) if os.path.isdir(DATASET_FOLDER) else []:
    if f.lower().endswith((".xlsx",".xls",".csv")):
        found_files.append(f)
found_files = sorted(found_files)
print("Found files:", found_files)

# if you want to explicitly set labels, edit label_map below.
# Default: try to auto-label by filename keywords.
label_map = {
    # "motor_off.xlsx": 0,
    # "motor_on.xlsx": 1,
    # "motor_on_nofan.xlsx": 2,
    # "motor_on_badfan.xlsx": 3
}

auto_labels = {}
if len(found_files) == 0:
    print("No dataset files found in folder. Please place Excel/CSV files in 'dataset' folder and re-run.")
else:
    for fn in found_files:
        low = fn.lower()
        if fn in label_map:
            auto_labels[fn] = label_map[fn]
        elif "off" in low:
            auto_labels[fn] = 0
        elif "nofan" in low or "no_fan" in low or "no-fan" in low:
            auto_labels[fn] = 2
        elif "bad" in low or "badfan" in low or "bad_fan" in low:
            auto_labels[fn] = 3
        elif "on" in low:
            auto_labels[fn] = 1
        else:
            # fallback: treat unknown as class 1 (motor on) but print a warning
            auto_labels[fn] = 1
            print(f"Warning: auto-labeling {fn} as class 1 (please set label_map explicitly if incorrect)")

print("Label mapping (file -> class):")
for k,v in auto_labels.items():
    print(" ", k, "->", v)

Looking for dataset files in: dataset
Found files: ['motor_off.csv', 'motor_on.csv', 'motor_on_badfan.csv', 'motor_on_nofan.csv']
Label mapping (file -> class):
  motor_off.csv -> 0
  motor_on.csv -> 1
  motor_on_badfan.csv -> 3
  motor_on_nofan.csv -> 2


In [13]:
# Cell 3: fit StandardScaler on concatenated raw sensor rows across all files
all_rows = []
for fn, lbl in auto_labels.items():
    path = os.path.join(DATASET_FOLDER, fn)
    try:
        if fn.lower().endswith(".csv"):
            df = pd.read_csv(path)
        else:
            df = pd.read_excel(path)
    except Exception as e:
        print("Skipping file (read error):", fn, ":", e)
        continue

    missing = [c for c in DATA_COLUMNS if c not in df.columns]
    if missing:
        print(f"Skipping {fn} — missing columns: {missing}")
        continue

    all_rows.append(df[DATA_COLUMNS].values)

if len(all_rows) == 0:
    print("No usable data rows found across files — cannot fit scaler or create windows. Check dataset files and DATA_COLUMNS.")
else:
    stacked = np.vstack(all_rows)
    scaler = StandardScaler()
    scaler.fit(stacked)
    joblib.dump(scaler, SCALER_PATH)
    print(f"Fitted scaler on {stacked.shape[0]} rows and saved to {SCALER_PATH}")

Fitted scaler on 216000 rows and saved to scaler_1.pkl


In [15]:
# Cell 4: create windows (X), labels (y), groups (group ids)
all_windows = []
all_labels = []
groups = []

if not os.path.exists(SCALER_PATH):
    print("Scaler not found, cannot produce standardized windows. Ensure Cell 3 ran successfully.")
else:
    scaler = joblib.load(SCALER_PATH)
    for fn, lbl in auto_labels.items():
        path = os.path.join(DATASET_FOLDER, fn)
        try:
            if fn.lower().endswith(".csv"):
                df = pd.read_csv(path)
            else:
                df = pd.read_excel(path)
        except Exception as e:
            print("Skipping file (read error):", fn, ":", e)
            continue

        # check columns
        if any(c not in df.columns for c in DATA_COLUMNS):
            print("Skipping file (missing expected cols):", fn)
            continue

        arr = df[DATA_COLUMNS].values
        if arr.shape[0] < WINDOW_SIZE:
            print(f"File too short for windows (len {arr.shape[0]} < {WINDOW_SIZE}): {fn} — skipping")
            continue

        scaled = scaler.transform(arr)
        for start in range(0, scaled.shape[0] - WINDOW_SIZE + 1, STRIDE):
            window = scaled[start:start+WINDOW_SIZE]
            all_windows.append(window)
            all_labels.append(lbl)
            groups.append(fn)   # group by filename

    if len(all_windows) == 0:
        print("No windows were created from dataset. Check your files, DATA_COLUMNS and WINDOW_SIZE.")
    else:
        X = np.array(all_windows, dtype=np.float32)   # (n_windows, WINDOW_SIZE, n_channels)
        y = np.array(all_labels, dtype=np.int32)
        groups = np.array(groups)
        print("Created windows:", X.shape, "Labels:", np.unique(y), "Groups:", len(np.unique(groups)))

Created windows: (6744, 64, 6) Labels: [0 1 2 3] Groups: 4


In [17]:
# Cell 5: create spectrogram images X_spec for each window (accel channels)
if 'X' not in globals():
    print("X not defined — skipping spectrogram creation. Ensure previous cells created X.")
else:
    # get spectrogram dims from one example
    f, t, Sxx = spectrogram(X[0, :, 0], fs=FS, nperseg=SPEC_NPERSEG, noverlap=SPEC_NOVERLAP)
    freq_bins = len(f)
    time_bins = len(t)
    n_windows = X.shape[0]
    X_spec = np.zeros((n_windows, freq_bins, time_bins, len(ACC_CHANNELS)), dtype=np.float32)

    for i in range(n_windows):
        for ch_idx, ch in enumerate(ACC_CHANNELS):
            col_idx = DATA_COLUMNS.index(ch)
            sig = X[i, :, col_idx]
            f, tt, Sxx = spectrogram(sig, fs=FS, nperseg=SPEC_NPERSEG, noverlap=SPEC_NOVERLAP)
            X_spec[i, :, :, ch_idx] = np.log1p(Sxx)
    print("Created X_spec:", X_spec.shape)

Created X_spec: (6744, 17, 3, 3)


In [19]:
# Cell 6: build hybrid model
if 'y' not in globals():
    print("y not defined — cannot build model.")
else:
    num_classes = len(np.unique(y))
    tf.keras.backend.clear_session()
    inp1 = layers.Input(shape=(WINDOW_SIZE, len(DATA_COLUMNS)), name='raw_input')
    x = layers.Conv1D(32, 11, padding='same', kernel_regularizer=regularizers.l2(L2_REG))(inp1)
    x = layers.BatchNormalization()(x); x = layers.Activation('relu')(x); x = layers.MaxPooling1D(2)(x)
    x = layers.Conv1D(64, 9, padding='same', kernel_regularizer=regularizers.l2(L2_REG))(x)
    x = layers.BatchNormalization()(x); x = layers.Activation('relu')(x); x = layers.MaxPooling1D(2)(x)
    x = layers.Conv1D(128, 5, padding='same', kernel_regularizer=regularizers.l2(L2_REG))(x)
    x = layers.BatchNormalization()(x); x = layers.Activation('relu')(x)
    x = layers.GlobalAveragePooling1D()(x); x = layers.Dropout(DROPOUT_RATE)(x)
    emb1 = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(L2_REG))(x)

    inp2 = layers.Input(shape=(X_spec.shape[1], X_spec.shape[2], X_spec.shape[3]), name='spec_input')
    yb = layers.Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(L2_REG))(inp2)
    yb = layers.BatchNormalization()(yb); yb = layers.Activation('relu')(yb); yb = layers.MaxPooling2D((2,1))(yb)
    yb = layers.Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(L2_REG))(yb)
    yb = layers.BatchNormalization()(yb); yb = layers.Activation('relu')(yb); yb = layers.MaxPooling2D((2,1))(yb)
    yb = layers.Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(L2_REG))(yb)
    yb = layers.BatchNormalization()(yb); yb = layers.Activation('relu')(yb)
    yb = layers.GlobalAveragePooling2D()(yb); yb = layers.Dropout(DROPOUT_RATE)(yb)
    emb2 = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(L2_REG))(yb)

    merged = layers.Concatenate()([emb1, emb2])
    merged = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(L2_REG))(merged)
    merged = layers.Dropout(0.4)(merged)
    out = layers.Dense(num_classes, activation='softmax')(merged)

    hybrid_model_template = models.Model(inputs=[inp1, inp2], outputs=out, name='hybrid_template')
    hybrid_model_template.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
                                  loss='categorical_crossentropy', metrics=['accuracy'])
    hybrid_model_template.summary()

Model: "hybrid_template"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ raw_input           │ (None, 64, 6)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spec_input          │ (None, 17, 3, 3)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 64, 32)    │      2,144 │ raw_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 17, 3, 32) │        896 │ spec_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 64, 32)    │        128 │ conv1d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 17, 3, 32) │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 64, 32)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 17, 3, 32) │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 32, 32)    │          0 │ activation[0][0]  │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 8, 3, 32)  │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 32, 64)    │     18,496 │ max_pooling1d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 8, 3, 64)  │     18,496 │ max_pooling2d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 64)    │        256 │ conv1d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 8, 3, 64)  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 32, 64)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 8, 3, 64)  │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_1     │ (None, 16, 64)    │          0 │ activation_1[0][… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 4, 3, 64)  │          0 │ activation_4[0][… │
│ (MaxPooling2D)      │                   │            │                 

 Total params: 190,308 (743.39 KB)

 Trainable params: 189,412 (739.89 KB)

 Non-trainable params: 896 (3.50 KB)

In [25]:
# Cell 7 (REPLACEMENT): Train the hybrid model with safe fold strategy (fixed compute_class_weight usage)
from sklearn.utils.class_weight import compute_class_weight

if 'X' not in globals() or 'X_spec' not in globals() or 'y' not in globals():
    print("Missing X, X_spec or y — cannot run training. Ensure previous cells executed successfully.")
else:
    # decide fold strategy
    unique_groups = np.unique(groups)
    if len(unique_groups) >= N_FOLDS:
        splitter = GroupKFold(n_splits=N_FOLDS)
        split_type = "GroupKFold"
    else:
        unique_labels, counts = np.unique(y, return_counts=True)
        if np.min(counts) >= N_FOLDS:
            splitter = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
            split_type = "StratifiedKFold"
        else:
            # fallback to a single train/Test split
            X_train, X_val, Xspec_train, Xspec_val, y_train, y_val = train_test_split(
                X, X_spec, y, test_size=0.2, stratify=y if len(np.unique(y)) > 1 else None, random_state=42)
            splitter = None
            split_type = "TrainTestSplit"
    print("Using split strategy:", split_type)

    # Training loop
    if split_type in ("GroupKFold", "StratifiedKFold"):
        fold = 0
        histories = []
        # use appropriate split call
        if split_type == "GroupKFold":
            split_gen = splitter.split(X, y, groups)
        else:
            split_gen = splitter.split(X, y)

        for train_idx, val_idx in split_gen:
            print(f"\n--- Fold {fold} ---")
            X_tr, X_val = X[train_idx], X[val_idx]
            Xspec_tr, Xspec_val = X_spec[train_idx], X_spec[val_idx]
            y_tr, y_val = y[train_idx], y[val_idx]

            # compute class weights correctly (keyword args required in newer sklearn)
            classes_in_tr = np.unique(y_tr)
            weights = compute_class_weight(class_weight='balanced', classes=classes_in_tr, y=y_tr)
            class_weight = {int(cls): float(w) for cls, w in zip(classes_in_tr, weights)}
            print("Class weights:", class_weight)

            # to categorical
            y_tr_cat = tf.keras.utils.to_categorical(y_tr, num_classes=num_classes)
            y_val_cat = tf.keras.utils.to_categorical(y_val, num_classes=num_classes)

            # new model per fold (clone from template)
            tf.keras.backend.clear_session()
            model = tf.keras.models.clone_model(hybrid_model_template)
            # compile fresh
            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
                          loss='categorical_crossentropy', metrics=['accuracy'])
            model.summary()

            # Callbacks
            cb_early = callbacks.EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True, verbose=1)
            cb_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=6, verbose=1, min_lr=1e-6)
            ckpt = f"hybrid_best_fold{fold}.keras"
            cb_ckpt = callbacks.ModelCheckpoint(ckpt, monitor='val_loss', save_best_only=True, verbose=0)

            history = model.fit(
                [X_tr, Xspec_tr], y_tr_cat,
                validation_data=([X_val, Xspec_val], y_val_cat),
                epochs=EPOCHS,
                batch_size=BATCH_SIZE,
                callbacks=[cb_early, cb_lr, cb_ckpt],
                class_weight=class_weight,
                verbose=2
            )
            histories.append(history.history)

            # Save fold model
            model.save(f"hybrid_model_fold{fold}.keras")
            print(f"Saved fold model hybrid_model_fold{fold}.keras and checkpoint {ckpt}")

            fold += 1
            if fold >= N_FOLDS:
                break

        # choose best fold (fold0) as canonical model if available
        if os.path.exists("hybrid_best_fold0.keras"):
            best = tf.keras.models.load_model("hybrid_best_fold0.keras")
            best.save(MODEL_SAVE_PATH)
            print("Saved canonical model:", MODEL_SAVE_PATH)
        else:
            print("No fold checkpoint found; training completed but no checkpoint present.")
    else:
        # Single split fallback training
        print("Training with single train/test split.")
        y_train_cat = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
        y_val_cat = tf.keras.utils.to_categorical(y_val, num_classes=num_classes)

        classes_in_tr = np.unique(y_train)
        weights = compute_class_weight(class_weight='balanced', classes=classes_in_tr, y=y_train)
        class_weight = {int(cls): float(w) for cls, w in zip(classes_in_tr, weights)}
        print("Class weights (single split):", class_weight)

        tf.keras.backend.clear_session()
        model = tf.keras.models.clone_model(hybrid_model_template)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
                      loss='categorical_crossentropy', metrics=['accuracy'])

        cb_early = callbacks.EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True, verbose=1)
        cb_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=6, min_lr=1e-6, verbose=1)
        ckpt = "hybrid_best_split.keras"
        cb_ckpt = callbacks.ModelCheckpoint(ckpt, monitor='val_loss', save_best_only=True, verbose=0)

        history = model.fit(
            [X_train, Xspec_train], y_train_cat,
            validation_data=([X_val, Xspec_val], y_val_cat),
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            callbacks=[cb_early, cb_lr, cb_ckpt],
            class_weight=class_weight,
            verbose=2
        )

        if os.path.exists(ckpt):
            best = tf.keras.models.load_model(ckpt)
            best.save(MODEL_SAVE_PATH)
            print("Saved model:", MODEL_SAVE_PATH)
        else:
            model.save(MODEL_SAVE_PATH)
            print("Saved model (no ckpt):", MODEL_SAVE_PATH)

Using split strategy: StratifiedKFold

--- Fold 0 ---
Class weights: {0: 1.0005563798219586, 1: 0.9998146775389177, 2: 0.9998146775389177, 3: 0.9998146775389177}


Model: "hybrid_template"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ raw_input           │ (None, 64, 6)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spec_input          │ (None, 17, 3, 3)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 64, 32)    │      2,144 │ raw_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 17, 3, 32) │        896 │ spec_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 64, 32)    │        128 │ conv1d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 17, 3, 32) │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 64, 32)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 17, 3, 32) │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 32, 32)    │          0 │ activation[0][0]  │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 8, 3, 32)  │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 32, 64)    │     18,496 │ max_pooling1d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 8, 3, 64)  │     18,496 │ max_pooling2d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 64)    │        256 │ conv1d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 8, 3, 64)  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 32, 64)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 8, 3, 64)  │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_1     │ (None, 16, 64)    │          0 │ activation_1[0][… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 4, 3, 64)  │          0 │ activation_4[0][… │
│ (MaxPooling2D)      │                   │            │                 

 Total params: 190,308 (743.39 KB)

 Trainable params: 189,412 (739.89 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ raw_input           │ (None, 64, 6)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spec_input          │ (None, 17, 3, 3)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 64, 32)    │      2,144 │ raw_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 17, 3, 32) │        896 │ spec_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 64, 32)    │        128 │ conv1d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 17, 3, 32) │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 64, 32)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 17, 3, 32) │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 32, 32)    │          0 │ activation[0][0]  │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 8, 3, 32)  │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 32, 64)    │     18,496 │ max_pooling1d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 8, 3, 64)  │     18,496 │ max_pooling2d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 64)    │        256 │ conv1d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 8, 3, 64)  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 32, 64)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 8, 3, 64)  │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_1     │ (None, 16, 64)    │          0 │ activation_1[0][… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 4, 3, 64)  │          0 │ activation_4[0][… │
│ (MaxPooling2D)      │                   │            │                 

 Total params: 190,308 (743.39 KB)

 Trainable params: 189,412 (739.89 KB)

 Non-trainable params: 896 (3.50 KB)

Epoch 1/60
85/85 - 3s - 36ms/step - accuracy: 0.6914 - loss: 0.7454 - val_accuracy: 0.5274 - val_loss: 1.0339 - learning_rate: 0.0010
Epoch 2/60
85/85 - 1s - 14ms/step - accuracy: 0.8556 - loss: 0.3814 - val_accuracy: 0.6914 - val_loss: 0.6601 - learning_rate: 0.0010
Epoch 3/60
85/85 - 1s - 14ms/step - accuracy: 0.8881 - loss: 0.2930 - val_accuracy: 0.6936 - val_loss: 0.8587 - learning_rate: 0.0010
Epoch 4/60
85/85 - 1s - 14ms/step - accuracy: 0.9012 - loss: 0.2720 - val_accuracy: 0.5727 - val_loss: 1.4956 - learning_rate: 0.0010
Epoch 5/60
85/85 - 1s - 14ms/step - accuracy: 0.9036 - loss: 0.2710 - val_accuracy: 0.5920 - val_loss: 1.7362 - learning_rate: 0.0010
Epoch 6/60
85/85 - 1s - 15ms/step - accuracy: 0.9372 - loss: 0.2039 - val_accuracy: 0.8353 - val_loss: 0.4023 - learning_rate: 0.0010
Epoch 7/60
85/85 - 1s - 14ms/step - accuracy: 0.9466 - loss: 0.1848 - val_accuracy: 0.7953 - val_loss: 0.7141 - learning_rate: 0.0010
Epoch 8/60
85/85 - 1s - 14ms/step - accuracy: 0.9461 - loss: 0

In [27]:
# Cell 8: quick evaluation on validation or full windows (if model exists)
if not os.path.exists(MODEL_SAVE_PATH):
    print("Model not found at", MODEL_SAVE_PATH, " — skip evaluation.")
else:
    model = tf.keras.models.load_model(MODEL_SAVE_PATH)
    # Evaluate on the full dataset windows (not ideal for real eval, but gives quick numbers)
    preds = model.predict([X, X_spec], verbose=0)
    pred_classes = np.argmax(preds, axis=1)
    acc = accuracy_score(y, pred_classes)
    f1 = f1_score(y, pred_classes, average='weighted')
    prec = precision_score(y, pred_classes, average='weighted', zero_division=0)
    rec = recall_score(y, pred_classes, average='weighted', zero_division=0)
    print(f"Quick eval on all windows -> acc: {acc:.3f}, f1: {f1:.3f}, prec: {prec:.3f}, rec: {rec:.3f}")


Quick eval on all windows -> acc: 0.705, f1: 0.703, prec: 0.730, rec: 0.705
